In [6]:
import psycopg2 as pg2
import pandas as pd
import numpy as np
import pickle

In [5]:
%%html
<style>
table {float:left}
</style>

In [3]:
with open('data/cs_df_M.pkl', 'rb') as f:
    df = pickle.load(f)

In [4]:
df.head()

,item_name,date_str,median_sell_price,quantity,est_release_str,days_since_release,gun_type,skin,quality,gun_bool,StatTrak,souvenir,keys,capsules,stickers,date,est_release,timestamp
0,Glock-18 | Weasel (Battle-Scarred),19 Aug 2016,6.193,135,19 Aug 2016,0,Glock-18,Weasel,Battle-Scarred,True,False,False,False,False,False,1.471583e+09,1.471583e+09,2016-08-19
1,Glock-18 | Weasel (Battle-Scarred),20 Aug 2016,5.103,166,19 Aug 2016,1,Glock-18,Weasel,Battle-Scarred,True,False,False,False,False,False,1.471669e+09,1.471583e+09,2016-08-20
2,Glock-18 | Weasel (Battle-Scarred),21 Aug 2016,4.721,142,19 Aug 2016,2,Glock-18,Weasel,Battle-Scarred,True,False,False,False,False,False,1.471756e+09,1.471583e+09,2016-08-21
3,Glock-18 | Weasel (Battle-Scarred),22 Aug 2016,4.302,98,19 Aug 2016,3,Glock-18,Weasel,Battle-Scarred,True,False,False,False,False,False,1.471842e+09,1.471583e+09,2016-08-22
4,Glock-18 | Weasel (Battle-Scarred),23 Aug 2016,3.898,125,19 Aug 2016,4,Glock-18,Weasel,Battle-Scarred,True,False,False,False,False,False,1.471928e+09,1.471583e+09,2016-08-23


## SQL Table Schemas:

#### ID Table:

item_id | item_name
---|---
int|str

#### Sales Table:

Could add days_since_release here or find a way to easily calculate it from a release date

item_id |date | price | quant | (days since release?)
---|---|---|---|---
int|float (unix)|float|int|(int?)

#### Details/Masking Table:

item_id | is_gun | is_capsule | is_key | is_sticker | is_case
---|---|---|---|---|---
int|bool|bool|bool|bool|bool

#### Guns Table:

item_id | gun_type | skin_name | quality | StatTrak | Souvenir
--|---|---|---|---|---
int|str|str|str|bool|bool

#### Release Date Table? Add to ID table? 

item_id | release_date
---|---
int|float (unix)

---

### Starting with the ID table and the sales table

In [7]:
# This created the table, doesn't need to be run again

# conn = pg2.connect(dbname='postgres', host='localhost')
# conn.autocommit = True
# cur = conn.cursor()
# cur.execute('CREATE DATABASE steam_capstone;')
# conn.close()

In [45]:
conn = pg2.connect(dbname='steam_capstone', host='localhost')
cur = conn.cursor()

#### Finding longest item name for `item_name` datatype (ended up using `text`)

In [12]:
max(map(len, df.item_name))

65

In [46]:
query = '''
        CREATE TABLE id (
            item_id SERIAL PRIMARY KEY, 
            item_name text
        );
        '''
cur.execute(query)

In [48]:
query = '''
        CREATE TABLE sales (
            item_id INTEGER REFERENCES id(item_id), 
            date date,
            price real,
            quantity integer
        );
        '''
cur.execute(query)

In [49]:
conn.commit()

In [50]:
cur.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cur.fetchall():
    print(table)

('sales',)
('id',)


In [51]:
conn.close()

### Inserting the data

In [54]:
def insert_item_names(names):
    """ insert multiple vendors into the vendors table  """
    sql = "INSERT INTO id(item_name) VALUES(%s)"
    conn = None
    try:
        # connect to the PostgreSQL database
        conn = pg2.connect(dbname='steam_capstone', host='localhost')
        # create a new cursor
        cur = conn.cursor()
        # execute the INSERT statement
        cur.executemany(sql, names)
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
    except (Exception, pg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [68]:
names = list(map(lambda x: (x,), df.item_name.unique()))

In [70]:
insert_item_names(names)

In [75]:
conn = pg2.connect(dbname='steam_capstone', host='localhost')
cur = conn.cursor()

In [76]:
cur.execute('SELECT * FROM id LIMIT 5;')
cur.fetchall()

[(1, 'Glock-18 | Weasel (Battle-Scarred)'),
 (2, 'PP-Bizon | Photic Zone (Minimal Wear)'),
 (3, 'SSG 08 | Ghost Crusader (Field-Tested)'),
 (4, 'Spectrum Case Key'),
 (5, 'Sticker | shroud (Foil) | Krakow 2017')]

In [77]:
conn.close()